<a href="https://colab.research.google.com/github/RochX/comp486-assignments/blob/main/assignment4/assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 4
## Xavier Silva

## Part 1

### Question 1
Use `read_csv` in `pandas` to read the CSV (`adm_data.csv`) file and assign it to a variable and print it.

In [6]:
from pathlib import Path
import pandas as pd
import urllib.request

# download the data if it is not downloaded
if not Path("adm_data.csv").is_file():
  # here I download the data from my personal git repo for this class instead of using Google Drive
  url = "https://raw.githubusercontent.com/RochX/comp486-assignments/main/assignment2/adm_data.csv"
  urllib.request.urlretrieve(url, "adm_data.csv")

In [7]:
adm_data=pd.read_csv('adm_data.csv')
adm_data

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
395,396,324,110,3,3.5,3.5,9.04,1,0.82
396,397,325,107,3,3.0,3.5,9.11,1,0.84
397,398,330,116,4,5.0,4.5,9.45,1,0.91
398,399,312,103,3,3.5,4.0,8.78,0,0.67


### Question 2
Split the data into a training set and a test set.

### Question 3
Using the training set, train a linear regression model to predict the chance of Admission to a university using `LinearRegression()` in Scikit-Learn, singular value
decomposition (SVD), Stochastic Gradient Descent, and Polynomial Regression with degree 2, then fill out the following table (show your full work). For each model:
1. What are the values of the model parameters ($\Theta$'s)?
2. Predict $\hat{y}$ for the new instance (`X_new`) where `X_new =[324, 112, 3, 2, 4.5, 8,1]`.
3. Calculate the score (using the test set).
4. Calculate the cost (`mean_squared_error`) (using the test set).
5. Write your observation about the results from the table.

In [ ]:
# TODO training code for each model here...

Table for question 3:

| Model/Technique | Thetas $\Theta$'s | Predict `X_new` | Score | `mean_squared_error` |
| --- | --- | --- | --- | --- |
| Linear regression using Scikit-Learn |
| Singular Value Decomposition (SVD) |
| Stochastic Gradient Descent |
| Polynomial Regression with degree 2 |

### Question 4
Which model would you adopt and why?

### Question 5
Train a decision tree model with `max_depth=2`, then predict the chance of Admission of `X_new`

In [ ]:
# TODO

## Part 2

### Question 1
Use `read_csv` in `pandas` to read the *training* set from the CSV ("Rock-Paper-
Scissor" training dataset) file and assign it to a variable and print it.

In [ ]:
# TODO

### Question 2
Use `read_csv` in `pandas` to read the *test* set from the CSV ("Rock-Paper-Scissor"
test dataset) file and assign it to a variable and print it.

In [ ]:
# TODO

### Question 3
Train a binary classifier to distinguish between just two classes, paper, and non-paper
using `LogisticRegression`, `LinearSVC (C = 0.01)`, and `LinearSVC (C = 100)`. For each model:
1. Predict $\hat{y}$ for some new instances (`X_new`) where (`X_new = Xtrain[88], Xtrain[2159], Xtrain[1346]`).
2. Find the probability that the model used to predict the class of an instance (if applicable).
3. Calculate the score (using the test set)
4. What are the `decision_function` values?
5. Write your observation about the results from the table.

In [ ]:
# TODO code this...

Table for question 3:

| Model/Technique | Predict `X_new` | Probability | `decision_function` | Score
| --- | --- | --- | --- | --- |
| `LogisticRegression` |
| `LinearSVC (C = 0.01)` |
| `LinearSVC (C = 100)` |

### Question 4
How can you predict the class of an instance using:
1. Probability (if applicable)
2. `decision_function`

## Part 3


TODO: Finish up writing rest of part 3 questions.